In [1]:
# PS2 - CE264
# GSI: Mustapha Harb - Mengqiao Yu
# Good Reference for this homework:
# https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb

# importing the requried libraries
from collections import OrderedDict    # For recording the model specification

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format
import warnings
warnings.filterwarnings("ignore")

# Load raw data

In [2]:
# reading the data file
data_path = '../data/raw/Air_Travel_Survey.csv'
data_01 = pd.read_csv(data_path, sep=",")

In [3]:
#look at the columns and the data
data_01.columns

Index([u'personID', u'gender', u'age', u'purpose', u'income', u'classTicket',
       u'payment', u'AA_FFP', u'CO_FFP', u'DL_FFP', u'B6_FFP', u'WN_FFP',
       u'UA_FFP', u'US_FFP', u'a1aircraft', u'a1departMAM', u'a1connections',
       u'a1travtime', u'a1arriveMAM', u'a1timediff', u'a1performance',
       u'a1fare', u'a1airline', u'a2aircraft', u'a2departMAM',
       u'a2connections', u'a2travtime', u'a2arriveMAM', u'a2timediff',
       u'a2performance', u'a2fare', u'a2airline', u'a1_AV', u'a2_AV',
       u'choice', u'choiceSituationID'],
      dtype='object')

In [4]:
data_01.head(20)

,personID,gender,age,purpose,income,classTicket,payment,AA_FFP,CO_FFP,DL_FFP,...,a2travtime,a2arriveMAM,a2timediff,a2performance,a2fare,a2airline,a1_AV,a2_AV,choice,choiceSituationID
0,1,1,5,1,10,3,1,1,1,1,...,895,1020,120,60,450,2,1,1,1,1
1,1,1,5,1,10,3,1,1,1,1,...,985,900,0,90,1050,7,1,1,1,2
2,1,1,5,1,10,3,1,1,1,1,...,980,1020,120,60,600,2,1,1,2,3
3,1,1,5,1,10,3,1,1,1,1,...,960,960,60,80,600,6,1,1,1,4
4,1,1,5,1,10,3,1,1,1,1,...,1130,900,0,90,900,7,1,1,2,5
5,1,1,5,1,10,3,1,1,1,1,...,1155,960,60,80,450,6,1,1,1,6
6,1,1,5,1,10,3,1,1,1,1,...,810,840,-60,70,900,8,1,1,1,7
7,1,1,5,1,10,3,1,1,1,1,...,1065,840,-60,70,1050,8,1,1,1,8
8,2,2,6,3,6,1,1,1,1,1,...,200,1010,60,60,600,3,1,1,1,9
9,2,2,6,3,6,1,1,1,1,1,...,275,1070,120,80,700,4,1,1,2,10


## Overview for binomial logit in python

### Step 0: Load the data
### Step 1: Define necessary variables and convert the data to long format.
### Step 2: Variable creations and transformations
### Step 3: Model specification
### Step 4: Run the model and analyze the results

## Step 1: Define necessary variables and convert the data to long format.
We need to specify five elements to construct a long format dataset in order to run the model under PyLogit.

(1.1) Individual related variables: the columns in the dataset that are specific to a given individual, regardless of what alternative is being considered. (e.g. gender)

(1.2) Alternative related variables (e.g. travel time).

(1.3) Altervative availabilities.

(1.4) Alternative and observation ids.

(1.5) The choice column.

In [5]:
# (1.1)
# Create the list of individual specific variables
ind_variables = data_01.columns.tolist()[:14]
print("ind_variables are:\n{}".format(ind_variables))

ind_variables are:
['personID', 'gender', 'age', 'purpose', 'income', 'classTicket', 'payment', 'AA_FFP', 'CO_FFP', 'DL_FFP', 'B6_FFP', 'WN_FFP', 'UA_FFP', 'US_FFP']


In [6]:
# (1.2)
# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative.

# {key1: value1, key2: value2}

alt_varying_variables = {u'aircraft_type': dict([(1, 'a1aircraft'),
                                                 (2, 'a2aircraft')]),
                          u'departure_time': dict([(1, 'a1departMAM'),
                                                   (2, 'a2departMAM')]),
                          u'connections': dict([(1, 'a1connections'),
                                                (2, 'a2connections')]),
                          u'travel_time': dict([(1, 'a1travtime'),
                                                (2, 'a2travtime')]),
                          u'arrival_time': dict([(1, 'a1arriveMAM'),
                                                 (2, 'a2arriveMAM')]),
                          u'time_diff': dict([(1, 'a1timediff'),
                                              (2, 'a2timediff')]),
                          u'performance': dict([(1, 'a1performance'),
                                                (2, 'a2performance')]),
                          u'fare': dict([(1, 'a1fare'),
                                         (2, 'a2fare')]),
                          u'airline': dict([(1, 'a1airline'),
                                            (2, 'a2airline')])}


In [7]:
# (1.3)
# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given alternative in the dataset.
availability_variables = {1: 'a1_AV', 2: 'a2_AV'}

In [8]:
# (1.4)
# Identify the alternative associated with each row.
custom_alt_id = "alternative_id"

# Create a custom id column that ignores the fact that this is a
# panel/repeated-observations dataset.
obs_id_column = "choiceSituationID"

In [9]:
# (1.5)
# Create a variable recording the choice column
choice_column = "choice"

In [10]:
# Perform the conversion to long-format
data_long =\
    pl.convert_wide_to_long(data_01,
                            ind_variables,
                            alt_varying_variables,
                            availability_variables,
                            obs_id_column,
                            choice_column,
                            new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
data_long.head(5).T

,0,1,2,3,4
choiceSituationID,1,1,2,2,3
alternative_id,1,2,1,2,1
choice,1,0,1,0,0
personID,1,1,1,1,1
gender,1,1,1,1,1
age,5,5,5,5,5
purpose,1,1,1,1,1
income,10,10,10,10,10
classTicket,3,3,3,3,3
payment,1,1,1,1,1


## Step 2: Variable creations and transformations

In [11]:
# Scale variables so the estimated coefficients are of similar magnitudes
# Scale travel time by 60 to convert raw units (minutes) to hours
data_long["travel_time_hrs"] = data_long["travel_time"] / 60.0

# Scale the fare column by 100 to convert raw units ($) to 100$
data_long["fare_100$"] = data_long["fare"] / 100.0

# Create dummy variables
data_long["fare_over500$"] = (data_long["fare_100$"] > 500).astype(int)

# data_long["interation_term"] =\
#     data_long["gender"] * data_long["legroom"]

# Create a human-readable airline column
data_long['airline_text'] =\
    data_long['airline'].map(
        {1: 'american',
         2: 'continental',
         3: 'delta',
         4: 'jet_blue',
         5: 'southwest',
         6: 'united',
         7: 'us_airways',
         8: 'other'}
    )

# Determine if the individual has a basic or elite FFP membership
# with each alternative's airline.
airline_text_to_ffp_column =\
    {'american': 'AA_FFP',
     'continental': 'CO_FFP',
     'delta': 'DL_FFP',
     'jet_blue': 'B6_FFP',
     'southwest': 'WN_FFP',
     'united': 'UA_FFP',
     'us_airways': 'US_FFP'
    }

data_long['basic_membership'] =\
    sum((data_long['airline_text'] == key) *
        (data_long[value] == 2)
        for key, value in airline_text_to_ffp_column.items()
       ).astype(int)

data_long['elite_membership'] =\
    sum((data_long['airline_text'] == key) *
        (data_long[value] == 3)
        for key, value in airline_text_to_ffp_column.items()
       ).astype(int)

# Determine if this alternative features a widebody aircraft
data_long['big_plane'] = (data_long['aircraft_type'] == 1).astype(int)

# Determine if this itinerary contains a red-eye flight, defined here
# as a flight departing between 8pm and 6am
data_long['red_eye'] =\
    ((data_long['departure_time'] < (6 * 60)) |
     (data_long['departure_time'] > (20 * 60))).astype(int)

## Step 3: Model specification

In [12]:
# specifying the utility equations

# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or or lists of lists of integers. Within a list,
#         or within the inner-most list, the integers should be the
#         alternative ID's of the alternative whose utility specification
#         the explanatory variable is entering. Lists of lists denote
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()

# Case A: alternative specific
basic_specification["travel_time_hrs"] = [1, 2]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (Alt 1)',
                                  'Travel Time, units:hrs (Alt 2)']

# Case B: generic: hw2
# basic_specification["travel_time_hrs"] = [[1, 2]]
# basic_names["travel_time_hrs"] = ['Travel Time, units:hrs']

# Case C: only for one
# basic_specification["travel_time_hrs"] = [1]
# basic_names["travel_time_hrs"] = ['Travel Time, units:hrs Alternative 1']

basic_specification["fare_100$"] = [1, 2]
basic_names["fare_100$"] =\
    ['Fare, units:hundredth (Alt 1)',
     'Fare, units:hundredth (Alt 2)']

basic_specification['red_eye'] = [1, 2]
basic_names['red_eye'] =\
    ['Red Eye (Alt 1), base:False',
     'Red Eye (Alt 2), base:False']

basic_specification['connections'] = [1, 2]
basic_names['connections'] =\
    ['Number of Connections (Alt 1)',
     'Number of Connections (Alt 2)']

basic_specification['big_plane'] = [1, 2]
basic_names['big_plane'] =\
    ['Widebody Aircraft (Alt 1), base:False',
     'Widebody Aircraft (Alt 2), base:False']

basic_specification['performance'] = [1, 2]
basic_names['performance'] =\
    ['On-Time Performance (%) (Alt 1)',
     'On-Time Performance (%) (Alt 2)']

basic_specification['basic_membership'] = [1, 2]
basic_names['basic_membership'] =\
    ['Basic FFP Membership (Alt 1), base:False',
     'Basic FFP Membership (Alt 2), base:False']

basic_specification['elite_membership'] = [1, 2]
basic_names['elite_membership'] =\
    ['Elite FFP Membership (Alt 1), base:False',
     'Elite FFP Membership (Alt 2), base:False']


#basic_specification["intercept"] = [1, 2]
# basic_names["intercept"] = ['ASC Alternative 1',
#                            'ASC Alternative 2']

## Now! Let's estimate the model and show the results

In [13]:
# Estimate the binary logit model (
air_travel_logit =\
    pl.create_choice_model(data=data_long,
                           alt_id_col=custom_alt_id,
                           obs_id_col=obs_id_column,
                           choice_col=choice_column,
                           specification=basic_specification,
                           model_type="MNL",
                           names=basic_names)

In [14]:
basic_names.values()

[['Travel Time, units:hrs (Alt 1)', 'Travel Time, units:hrs (Alt 2)'],
 ['Fare, units:hundredth (Alt 1)', 'Fare, units:hundredth (Alt 2)'],
 ['Red Eye (Alt 1), base:False', 'Red Eye (Alt 2), base:False'],
 ['Number of Connections (Alt 1)', 'Number of Connections (Alt 2)'],
 ['Widebody Aircraft (Alt 1), base:False',
  'Widebody Aircraft (Alt 2), base:False'],
 ['On-Time Performance (%) (Alt 1)', 'On-Time Performance (%) (Alt 2)'],
 ['Basic FFP Membership (Alt 1), base:False',
  'Basic FFP Membership (Alt 2), base:False'],
 ['Elite FFP Membership (Alt 1), base:False',
  'Elite FFP Membership (Alt 2), base:False']]

In [15]:
# Specify the initial values and method for the optimization.
# 4 being the total number of parameters to be estimated
num_params = sum(map(len, basic_names.values()))
initial_values = np.zeros(num_params)

air_travel_logit.fit_mle(initial_values)

Log-likelihood at zero: -4,868.6658
Initial Log-likelihood: -4,868.6658
Estimation Time for Point Estimation: 0.06 seconds.
Final log-likelihood: -3,846.7183


In [16]:
# Look at the estimation results
air_travel_logit.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                7,024
Model:             Multinomial Logit Model   Df Residuals:                    7,008
Method:                                MLE   Df Model:                           16
Date:                     Sun, 05 Apr 2020   Pseudo R-squ.:                   0.210
Time:                             18:55:35   Pseudo R-bar-squ.:               0.207
AIC:                             7,725.437   Log-Likelihood:             -3,846.718
BIC:                             7,835.150   LL-Null:                    -4,868.666
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Travel Time, units:hrs (Alt 1)              -0.2245      0.036     -6.262      0.000      -0.295      -0.154
Travel Time, units:hrs (Alt 2)              -0.2282      0.035     -6.431      0.000      -0.298      -0.159
Fare, units:hundredth (Alt 1)               -0.4869      0.021    -22.899      0.000      -0.529      -0.445
Fare, units:hundredth (Alt 2)               -0.5468      0.023    -24.257      0.000      -0.591      -0.503
Red Eye (Alt 1), base:False                 -0.2950      0.083     -3.546      0.000      -0.458      -0.132
Red Eye (Alt 2), base:False                 -0.1931      0.083     -2.322      0.020      -0.356      -0.030
Number of Connections (Alt 1)               -0.6254      0.064     -9.770      0.000      -0.751      -0.500
Number of Connections (Alt 2)               -0.6014      0.063     -9.540      0.000      -0.725      -0.478
Widebody Aircraft (Alt 1), base:False       -0.6020      0.101     -5.987      0.000      -0.799      -0.405
Widebody Aircraft (Alt 2), base:False       -0.7665      0.103     -7.428      0.000      -0.969      -0.564
On-Time Performance (%) (Alt 1)              0.0168      0.002      9.161      0.000       0.013       0.020
On-Time Performance (%) (Alt 2)              0.0169      0.002      9.215      0.000       0.013       0.020
Basic FFP Membership (Alt 1), base:False     0.4656      0.066      7.029      0.000       0.336       0.595
Basic FFP Membership (Alt 2), base:False     0.5321      0.066      8.018      0.000       0.402       0.662
Elite FFP Membership (Alt 1), base:False     0.5951      0.124      4.795      0.000       0.352       0.838
Elite FFP Membership (Alt 2), base:False     0.8789      0.125      7.033      0.000       0.634       1.124
============================================================================================================
"""

# Prepare oneself for model checking

To use the interactive model checking demonstration, there are two options.

1. Students can clone or download the `ce264_march10th_2020` branch of the `check-yourself` github repo, place their notebook into this directory, and directly make use of the model checking functions in the `src.visiualization.predictive_viz` module.

2. Students can copy the function below, run it locally on their computer, and upload the resulting file to Binder for use there.

In [17]:
air_travel_logit.specification

OrderedDict([('travel_time_hrs', [1, 2]),
             ('fare_100$', [1, 2]),
             ('red_eye', [1, 2]),
             ('connections', [1, 2]),
             ('big_plane', [1, 2]),
             ('performance', [1, 2]),
             ('basic_membership', [1, 2]),
             ('elite_membership', [1, 2])])

In [18]:
air_travel_logit.name_spec

OrderedDict([('travel_time_hrs',
              ['Travel Time, units:hrs (Alt 1)',
               'Travel Time, units:hrs (Alt 2)']),
             ('fare_100$',
              ['Fare, units:hundredth (Alt 1)',
               'Fare, units:hundredth (Alt 2)']),
             ('red_eye',
              ['Red Eye (Alt 1), base:False', 'Red Eye (Alt 2), base:False']),
             ('connections',
              ['Number of Connections (Alt 1)',
               'Number of Connections (Alt 2)']),
             ('big_plane',
              ['Widebody Aircraft (Alt 1), base:False',
               'Widebody Aircraft (Alt 2), base:False']),
             ('performance',
              ['On-Time Performance (%) (Alt 1)',
               'On-Time Performance (%) (Alt 2)']),
             ('basic_membership',
              ['Basic FFP Membership (Alt 1), base:False',
               'Basic FFP Membership (Alt 2), base:False']),
             ('elite_membership',
              ['Elite FFP Membership (Alt 1), base:

In [19]:
air_travel_logit.cov

,"Travel Time, units:hrs (Alt 1)","Travel Time, units:hrs (Alt 2)","Fare, units:hundredth (Alt 1)","Fare, units:hundredth (Alt 2)","Red Eye (Alt 1), base:False","Red Eye (Alt 2), base:False",Number of Connections (Alt 1),Number of Connections (Alt 2),"Widebody Aircraft (Alt 1), base:False","Widebody Aircraft (Alt 2), base:False",On-Time Performance (%) (Alt 1),On-Time Performance (%) (Alt 2),"Basic FFP Membership (Alt 1), base:False","Basic FFP Membership (Alt 2), base:False","Elite FFP Membership (Alt 1), base:False","Elite FFP Membership (Alt 2), base:False"
"Travel Time, units:hrs (Alt 1)",1.285027e-03,1.223734e-03,0.000037,0.000047,-5.324168e-04,-4.158455e-04,-0.001622,-0.001481,0.000144,-0.000043,-2.497472e-06,5.519192e-07,-0.000038,-0.000052,0.000076,2.891295e-05
"Travel Time, units:hrs (Alt 2)",1.223734e-03,1.258900e-03,0.000043,0.000035,-4.055485e-04,-5.321192e-04,-0.001508,-0.001559,-0.000052,0.000142,-1.743179e-07,-2.018563e-06,-0.000037,-0.000042,0.000093,-2.771433e-06
"Fare, units:hundredth (Alt 1)",3.695636e-05,4.346317e-05,0.000452,0.000417,7.513642e-05,7.141216e-05,0.000085,0.000078,0.000073,0.000045,-3.319001e-06,-1.772483e-06,-0.000049,-0.000080,-0.000098,-9.344919e-05
"Fare, units:hundredth (Alt 2)",4.691280e-05,3.536979e-05,0.000417,0.000508,1.037480e-04,9.150830e-05,0.000094,0.000099,0.000062,0.000067,-1.667041e-06,-3.794660e-06,-0.000061,-0.000086,-0.000041,-1.446442e-04
"Red Eye (Alt 1), base:False",-5.324168e-04,-4.055485e-04,0.000075,0.000104,6.921783e-03,3.487238e-03,-0.000116,0.000228,-0.000026,-0.000010,6.391384e-07,-2.486256e-06,-0.000010,-0.000075,-0.000005,-2.738380e-04
"Red Eye (Alt 2), base:False",-4.158455e-04,-5.321192e-04,0.000071,0.000092,3.487238e-03,6.912260e-03,0.000088,-0.000136,-0.000064,0.000033,1.184554e-06,9.144851e-07,-0.000072,-0.000091,0.000043,-2.103611e-04
Number of Connections (Alt 1),-1.621955e-03,-1.507590e-03,0.000085,0.000094,-1.157125e-04,8.774982e-05,0.004097,0.002226,-0.000144,0.000112,-7.088942e-06,-1.988052e-06,-0.000090,-0.000071,-0.000290,-2.604503e-04
Number of Connections (Alt 2),-1.480700e-03,-1.558764e-03,0.000078,0.000099,2.282720e-04,-1.358523e-04,0.002226,0.003974,0.000199,0.000002,-1.908223e-06,-8.256080e-06,-0.000035,-0.000082,-0.000234,-1.479631e-04
"Widebody Aircraft (Alt 1), base:False",1.439482e-04,-5.201510e-05,0.000073,0.000062,-2.576652e-05,-6.399452e-05,-0.000144,0.000199,0.010109,0.001527,-1.392601e-05,9.134098e-06,-0.000172,-0.000064,0.000051,-5.698801e-05
"Widebody Aircraft (Alt 2), base:False",-4.283997e-05,1.424334e-04,0.000045,0.000067,-1.004778e-05,3.266409e-05,0.000112,0.000002,0.001527,0.010649,4.595994e-06,-1.909180e-05,0.000011,-0.000172,-0.000063,-1.233372e-04


In [ ]:
def make_directory_if_necessary(filename):
    """
    Creates the directories for the given file if they do not already exit.
    """
    import os
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        # Guard against race condition where directories have been created
        # between the os.path.exists call and the os.makedirs call.
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise
    return None

def package_model_for_binder(df, fitted_model, temp_dir='./temp'):
    import os
    import json
    import shutil
    ####
    # Save all needed objects in the temporary directory
    ####
    # Create a path for the asymptotic covariance matrix
    cov_path = os.path.join(temp_dir, 'cov.csv')
    # Create the temporary directory if needed
    make_directory_if_necessary(cov_path)
    # Save the asymptotic covariance matrix
    fitted_model.cov.to_csv(cov_path, index=True)

    # Save the dataframe used to estimate the model
    df_path = os.path.join(temp_dir, 'df.csv')
    df.to_csv(df_path, index=False)

    # Save the estimated parameters
    param_path = os.path.join(temp_dir, 'params.csv')
    fitted_model.params.to_csv(param_path)

    # Save the model specification and name dictionaries
    spec_path = os.path.join(temp_dir, 'spec.json')
    with open(spec_path, 'wb') as fpath:
        json.dump(air_travel_logit.specification, fpath)

    name_path = os.path.join(temp_dir, 'names.json')
    with open(name_path, 'wb') as fpath:
        json.dump(air_travel_logit.name_spec, fpath)

    # Zip the temporary directory
    shutil.make_archive('temp', 'zip', root_dir=temp_dir)
    return None

def unpack_on_binder(zip_file_path, temp_dir='./temp'):
    import os
    import json
    import shutil
    import pandas as pd
    from collections import OrderedDict

    # Unpack the zip file to the temporary directory.
    shutil.unpack_archive(zip_file_path, temp_dir)
    # Load the needed objects from the temporary directory
    cov_path = os.path.join(temp_dir, 'cov.csv')
    cov_df = pd.read_csv(cov_path)

    df_path = os.path.join(temp_dir, 'df.csv')
    df = pd.read_csv(df_path)

    param_path = os.path.join(temp_dir, 'params.csv')
    params = pd.read_csv(param_path)

    spec_path = os.path.join(temp_dir, 'spec.json')
    with open(spec_path, 'rb') as f:
        spec = json.load(f, object_pairs_hook=OrderedDict)

    name_path = os.path.join(temp_dir, 'names.json')
    with open(name_path, 'rb') as f:
        name_spec = json.load(f, object_pairs_hook=OrderedDict)

    # Package the loaded objects into a dictionary for return
    results_dict =\
        {'cov_df': cov_df,
         'df': df,
         'param_series': params,
         'spec_dict': spec,
         'name_spec_dict': name_spec}

    # Return the created dictionary
    return results_dict




In [21]:
# Test out the function for packaging results for binder
package_model_for_binder(data_long, air_travel_logit)